In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pathresults = "../results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","
import re

In [24]:
def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst
def get_last_line_as_numpy_array(file_path, dtype = float):
    with open(pathresults +file_path, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

def list_files_with_prefix(folder_path, prefix):
    file_names = []
    for file_name in os.listdir(pathresults+folder_path):
        if file_name.startswith(prefix):
            file_names.append(file_name)
    return file_names
def getCylIdx(folder_path):
    namelist = list_files_with_prefix(folder_path+"cyl_val/", "Cyl_watercontent_")
    idxlist = np.full(len(namelist),-1,dtype = int)
    for nn, nl in enumerate(namelist):
        match = re.search(r'\d+', nl)
        number_str = match.group()
        number_int = int(number_str)
        idxlist[nn] = number_int
    return idxlist
# cylinder max konz (per cell)
def getData_(cid,gId,path2file):
    fileName = "cyl_val/Cyl_cellVol_"+str(gId)+".txt"
    cVol = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
    if cid == volIdx:
        return sum(cVol)
    elif cid <= 9:
        if cid <= 2:# == 0:
            fileName = "cyl_val/Cyl_watercontent_"+str(gId)+".txt"
            theta = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
            cVol *= theta#cm3 scv to cm3 water
            if cid == 0:
                return sum(cVol)
        fileName = "cyl_val/Cyl_content"+str(cid)+"_"+str(gId)+".txt"
        Q1 = get_last_line_as_numpy_array(path2file+fileName, dtype = float)
        konz = sum(Q1)/sum(cVol) #mol/cm3 or cm3/cm3
        return konz
    raise Exception

numPset = 99
nKonz = 9
dictXYZ = {9:'X',10:'Y',11:'Z'}
volIdx=12
lenIdx=13
stIdx=14
orgIdId=15
relLenId=16

nToGet = relLenId+1 #elements + 3d coordinates of y , node+vol+len+st 

exceptPset =[12, 13, 20, 22, 32, 39, 44, 61, 66, 67, 91]
allPSets = [ i for i in range(numPset) if i not in exceptPset]

def getData():
    
    GiniAll = [[[] for i in range(numPset)] for ii in range(nToGet)]
    for pSet in allPSets:
        print(pSet,end =", ")

        #path2file = 'none_55_'+str(pSet)+'_10to11_20mn_0s_5/'
        path2file = 'noCss1none_1440_'+str(pSet)+'_10to11_20mn_0s_5/'
        
        rr = getCylIdx(path2file)
        rr.sort()
        print('(',len(rr),')',end =", ")

        for cid in range(nKonz):
            Ginits = np.array([getData_( cid, gId,path2file) for gId in rr])          
            GiniAll[cid][pSet] = Ginits
        for cid in range(nKonz,volIdx):
            nodes_ = get_last_line_as_numpy_array(path2file+"nodes_"+dictXYZ[cid]+".txt", dtype =float)
            GiniAll[cid][pSet] = nodes_[rr+1]#seg idx to y-idx
        #cid = nToGet -1
        Ginits = np.array([getData_( volIdx, gId,path2file) for gId in rr])
        GiniAll[volIdx][pSet] = Ginits
        lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        GiniAll[lenIdx][pSet] = lens[rr]
        sts = get_last_line_as_numpy_array(path2file+"subTypes"+".txt", dtype =float)
        assert len(sts) == len(lens)
        GiniAll[stIdx][pSet] = sts[rr]
        
        orgidPerNode = get_last_line_as_numpy_array(path2file+"orgidPerNode.txt", dtype =int)
        globalNodeId = get_last_line_as_numpy_array(path2file+"globalNodeId.txt", dtype =int)
        parentNidx = get_last_line_as_numpy_array(path2file+"parentNidx.txt", dtype =int)
        ot_orgs = get_last_line_as_numpy_array(path2file+"ot_orgs.txt", dtype =int)
        id_orgs = get_last_line_as_numpy_array(path2file+"id_orgs.txt", dtype =int)
        id_roots = id_orgs[np.where(ot_orgs==2)]
        
        org2basenode = dict(zip(id_orgs, parentNidx))
        #org2ot = dict(zip(id_orgs, ot_orgs))
        isRoot = np.array([oo in id_roots for oo in orgidPerNode])
        
        orgidPerRoot = orgidPerNode[np.where(isRoot)]
        globalNodeIdroot =  globalNodeId[np.where(isRoot)]        
        segIdxRoot = np.array([rr for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]]) -1
        orgIdxRoot = np.array([orgidPerRoot[_] for _, rr in enumerate(globalNodeIdroot) if rr != org2basenode[orgidPerRoot[_]]])
        #segIdx2ot = dict(zip(segIdxRoot,orgIdxRoot))
        
        relLens = np.zeros(len(segIdxRoot))
        #lens = get_last_line_as_numpy_array(path2file+"length_Segs"+".txt", dtype =float)
        for myorg in orgIdxRoot:
            thensegs = np.where(orgIdxRoot == myorg)#segIdxRoot[] 
            lenSegs = GiniAll[lenIdx][pSet]
            relLens[thensegs] = np.cumsum(lenSegs[thensegs])
            
            
        GiniAll[orgIdId][pSet] = orgIdxRoot
        GiniAll[relLenId][pSet] = relLens

    return GiniAll #cid pSet rr
    

In [12]:
pSet = 0
path2file = 'noCss1none_1440_'+str(pSet)+'_10to11_20mn_0s_5/'
        
rr = getCylIdx(path2file)
rr.sort()
print('(',len(rr),')',end =", ")

        

( 238 ), 

In [17]:

exceptPset =[12, 13, 20, 22, 32, 39, 44, 61, 66, 67, 91]
allPSets = [ i for i in range(numPset) if i not in exceptPset]

In [13]:
GiniAll = getData()

0, ( 238 ), 1, ( 234 ), 2, ( 237 ), 3, ( 237 ), 4, ( 240 ), 5, ( 216 ), 6, ( 234 ), 7, ( 244 ), 8, ( 233 ), 9, ( 236 ), 10, ( 236 ), 11, ( 236 ), 14, ( 239 ), 15, ( 236 ), 16, ( 242 ), 17, ( 230 ), 18, ( 236 ), 19, ( 238 ), 21, ( 216 ), 23, ( 237 ), 24, ( 242 ), 25, ( 241 ), 26, ( 236 ), 27, ( 245 ), 28, ( 233 ), 29, ( 236 ), 30, ( 236 ), 31, ( 238 ), 33, ( 234 ), 34, ( 249 ), 35, ( 244 ), 36, ( 236 ), 37, ( 237 ), 38, ( 236 ), 40, ( 221 ), 41, ( 242 ), 42, ( 237 ), 43, ( 244 ), 45, ( 237 ), 46, ( 237 ), 47, ( 244 ), 48, ( 234 ), 49, ( 244 ), 50, ( 242 ), 51, ( 244 ), 52, ( 239 ), 53, ( 221 ), 54, ( 234 ), 55, ( 244 ), 56, ( 244 ), 57, ( 220 ), 58, ( 240 ), 59, ( 237 ), 60, ( 244 ), 62, ( 249 ), 63, ( 236 ), 64, ( 240 ), 65, ( 234 ), 68, ( 234 ), 69, ( 237 ), 70, ( 244 ), 71, ( 236 ), 72, ( 242 ), 73, ( 244 ), 74, ( 206 ), 75, ( 244 ), 76, ( 236 ), 77, ( 234 ), 78, ( 242 ), 79, ( 237 ), 80, ( 240 ), 81, ( 236 ), 82, ( 238 ), 83, ( 244 ), 84, ( 230 ), 85, ( 244 ), 86, ( 244 ), 87, ( 239

In [131]:
#assert np.array(GiniAll).shape == (nToGet, numPset, 232) # some have a couple of segments more

/tmp/ipykernel_1126/1273250247.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert np.array(GiniAll).shape == (nToGet, numPset, 232) #all same shape


AssertionError: 

In [14]:
np.array(GiniAll).shape

/tmp/ipykernel_498436/2873905747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(GiniAll).shape


(17, 99)

In [28]:
# get param set
paramsets =  pd.read_csv('../output_random_rows.csv')
paramsets = paramsets.drop(paramsets.columns.tolist()[-1],axis=1)
paramsets['k_C,S'] /=paramsets['theta']
paramsets['k_O,S'] /=paramsets['theta']
paramsets.drop(labels='theta', inplace=True, axis=1)
outputvalsname =  ['wat','cs','cl','coa','cod','cca','ccd','css2','co2','yX','yY','yZ','vol','lenSeg','st','orgId','relLen']
assert len(outputvalsname) == nToGet 
column_names = outputvalsname + paramsets.columns.tolist()+['pSet']
df = pd.DataFrame(columns=column_names)

for pset in allPSets:#range(numPset):
    N = GiniAll[0][pset].shape[0]
    #assert N == 232
    rows_to_duplicate_df = pd.DataFrame(paramsets.iloc[pset]).T
    duplicated_rows_df = rows_to_duplicate_df.loc[np.repeat(rows_to_duplicate_df.index.values, N)]
    # Duplicate the specific rows N times
    #print('init dupl row shape',duplicated_rows_df.shape)
    for jj, outVn in enumerate(outputvalsname):
        duplicated_rows_df[outVn] = GiniAll[jj][pset]
        #print('adding',jj,outVn,duplicated_rows_df.shape,len(GiniAll[jj][pset]))
    duplicated_rows_df['pSet'] = pset
    
    path2file = 'noCss1none_1440_'+str(pset)+'_10to11_20mn_0s_5/'
    simTime = get_last_line_as_numpy_array(path2file+"totalComputetime.txt", dtype =float)[0]
    
    duplicated_rows_df['simTime'] = simTime
    # Concatenate the original DataFrame with the duplicated specific rows
    df = pd.concat([df, duplicated_rows_df], ignore_index=True)
df['co'] = df['coa']+df['cod']
df['cc'] = df['cca']+df['ccd']

In [30]:
df['simTime'] 

0        22865.345728
1        22865.345728
2        22865.345728
3        22865.345728
4        22865.345728
             ...     
20896    28032.934171
20897    28032.934171
20898    28032.934171
20899    28032.934171
20900    28032.934171
Name: simTime, Length: 20901, dtype: float64

In [29]:
df.to_pickle('./cyl10to11.pkl')

In [ ]:

path2file = 'noCss1none_1440_'+str(0)+'_10to11_20mn_0s_5/'
simTime = get_last_line_as_numpy_array(path2file+"totalComputetime.txt", dtype =float)[0]

In [ ]:
simTime

In [31]:
min(df['simTime'] )

9925.080730255693